import libraries

In [1]:
import numpy
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from pathlib import Path
import math
from math import pi
from math import sqrt
import statistics
import scipy.integrate
from scipy.signal import find_peaks
from scipy.signal import argrelmin
from numpy import trapz
from scipy.integrate import cumtrapz

user inputs

In [2]:
# SETUP VARIABLES - USER INPUTS
BD = 2 #Bluedrop file is from 
fileNum = '0021' # write the bin file number you want to analyze (do not include 'bLog' or '.bin')
soiltype = "s" #s = sand, c=clay, m=mixed, u=unknown
atype = 'm'  # m = mantle area (best for sands), p = projected area (best for clays)
tiptype = 'c'  # c = cone, p = parabolic, b = blunt
#offset = 1 # this value is subtracted from the accelerometer readings
droptype = 'w' #w = water, #a = air
sign = "uk" #enter an effective unit weight value in kg/m^3 or "uk" if unknown 
# paste the filepath to the folder where the BD data is stored
binFilepath = Path("H:/.shortcut-targets-by-id/1aF9t2aiRGWTftJMZFAOBixqvQniFBjnb/Duck  2023/Data/Intertidal/BlueDrop, Samples & Moisture Gage/07March23/BD2 3.7.23 - Drops Only")
#paste the filepath to an excel file that the analysis results will be printed in
outputPath = Path("H:/.shortcut-targets-by-id/1aF9t2aiRGWTftJMZFAOBixqvQniFBjnb/Duck  2023/Data/Intertidal/BlueDrop, Samples & Moisture Gage/07March23/BlueDrop Data Processing March 7 2023.xlsx") #  Path to pre-existing Excel File
plotPath = Path("H:/.shortcut-targets-by-id/1aF9t2aiRGWTftJMZFAOBixqvQniFBjnb/Duck  2023/Data/Intertidal/BlueDrop, Samples & Moisture Gage/07March23/BD2 3.8.23 - Drops Only/Analysis Figures")


set functions

In [3]:

# FUNCTIONS FOR USE IN ANALYSIS
def masslength(tiptype): #sets the mass and length of the pentrometer based on the tip
    global mass
    global length
    if tiptype == 'c':
        mass = 7.71 #kg
        length = 8.48-0.93 #originally 7.87, 7.57 for perfect 60 deg, 8.48 measured - .93 measured 90 deg
    elif tiptype == 'p':
        mass = 9.15
        length = 8.26
    elif tiptype == 'b':
        mass = 10.30
        length = 8.57

def dropstartend(peak): #after locating the peaks, this function chops the minute long file into a smaller segment immediately before and after the peak
    global dropstart
    global dropend
    if peak <= 1500:
        dropstart = 1
        dropend = peak + 500
    elif peak > 119500:
        dropstart = peak - 1500
        dropend = 120000
    else:
        dropstart = peak - 1500
        dropend = peak + 500

def accPick(dg, d): #this function picks the smallest accelerometer that's not maxed out to perform the integration on
    maxAcc = dg["250g (g)"].max()
    global acc
    global accName
    global accNameg
    global accg
    if maxAcc < 5: #- offset:
        if dg["2g (g)"].max() < 1.8: #offset:  # does an extra check for the 2g because of noise
            acc = dfCal["2g (m/s^2)"]
            accg = dfCalg["2g (g)"]
            accName = "2g (m/s^2)"
            accNameg = "2g (g)"
        else:
            acc = dfCal["18g (m/s^2)"]
            accg = dfCalg["18g (g)"]
            accName = "18g (m/s^2)"
            accNameg = "18g (g)"
    elif maxAcc < 18: #- offset
        acc = dfCal["18g (m/s^2)"]
        accg = dfCalg["18g (g)"]
        accName = "18g (m/s^2)"
        accNameg = "18g (g)"
    elif maxAcc < 50: #- offset:
        acc = dfCal["50g (m/s^2)"]
        accg = dfCalg["50g (g)"]
        accName = "50g (m/s^2)"
        accNameg = "50g (g)"
    else:
        acc = dfCal["250g (m/s^2)"]
        accg = dfCalg["50g (g)"]
        accName = "250g (m/s^2)"
        accNameg = "250g (g)"

def findchangepts(): #This function picks the moment that the BD impacts the ground (not in use currently)
    global drop
    jlist = list()
    global jindex
    print("finding start of drop...")
    for i in range(4,len(accg)-4):
        p1 = 1
        #print(p1)
        p2 = i
        #print(p2)
        p3 = len(accg)
        #print(p3)
        sample1 = list(accg[p1:p2-1])
        #print(sample1)
        sample2 = list(accg[p2:p3])
        #print(sample2)
        stat1 = math.log(statistics.variance(sample1))
        stat2 = math.log(statistics.variance(sample2))
        #print(stat1)

        j1 = (i-1)*stat1
        j2 = ((len(accg)-1)-i+1)*stat2
        j = j1+j2
        #print(j)
        jlist.append(j)

    drop = min(jlist)
    #print("drop is")
    #print(drop)
    jlist = np.array(jlist)
    #print(jlist)
    #print(jlist.size)
    jlist = np.insert(jlist, 0, (0,0,0,0)) #reshape to match up with dataframe d
    jlist = np.append(jlist, (0,0,0,0)) #reshape to match up with dataframe d
    #print(jlist.size) #should be 2000
    jindex = np.where(jlist==drop) #finds the index of the drop start
    jindex = int(jindex[0]) #converts the index into a number from a tuple
    print("Jlist: ", jlist)

def finddropend(n): #finds the location after the peak where the deceleration is 1 
    global penend
    below0list = list()
    #for i in range(dropstart+jindex, dropend, 1):
    for i in range(peak, dropend, 1):
        if accg[i] < 1: #the offset is applied later in the code
            penend = i 
            below0list = np.append(below0list, penend)
    penend=int(min(below0list))

def integration(d, accint): #integrates the deceleration data to solve for velocity and penetration depth
    global vel
    global maxvel
    global dep
    global maxdep
    #accint = acc[penstart:penend]
    #print(len(d))
    #print(len(accint))
    vel = scipy.integrate.cumtrapz(accint, x=d["Time (s)"])  # solves for velocity
    vel = np.array(vel)
    vel = numpy.insert(vel, 0, 0) #not sure what to insert here, but it makes it the right size
    vel = np.flip(vel)
    maxvel = vel.max()
    maxvel = round(maxvel,1)
    dep = scipy.integrate.cumtrapz(vel, x=d["Time (s)"])  # solves for penetration depth
    dep = numpy.insert(dep, 0, 0)  # not sure what to insert here, but it makes it the right size
    maxdep = dep.max()
    maxdep = round(maxdep,4)
    d.insert(9, "Velocity (m/s)", vel)
    d.insert(10, "Penetration Depth (m)", dep)

def peakpick():
    global penstart
    fig, (ax1) = plt.subplots(1)
    plt.plot(dfCalg['Count'], dfCalg[accNameg], label=accNameg)
    plt.plot(dfCalg['Count'], dfCalg['2g (g)'], label='2g')

    ax1.set_xlim(left=dropstart, right=dropend)
    ax1.legend()
    ax1.set(ylabel="Deceleration (g)")
    ax1.set(xlabel="Time")
    ax1.set_title('Double-click start of drop #' + str(a))
    startendpt = plt.ginput(2, 0)
    pentimes = []
    for t in startendpt:
        for x in t:
            pentimes.append(x)

    penstart = int(pentimes[0])
    print("start of penetration: ", penstart)

def CheckingFunction(): # (to be used with auto peak finding) This function is added to check the values of the start and end of the drop if the autofinding function doesn't work

    global jindex
    global num1
    
    xvalues=np.linspace(0,len(dg)-1,len(dg))
    plt.figure(num=1)
    temp_dg=dg['250g (g)']
    plt.plot(xvalues,temp_dg)
    plt.grid(visible=True,which='both')
    plt.show()
    print ('The start of the drop is', jindex)
    print ('The end of the drop is', num1)
    print('Are the start and end of the drop correct?(y/n)')
    drop_input=input()
    if drop_input=='n':   
        peakpick()
        jindex = penstart
        num1 = penend

def areafind(): #finds the embedded area based on the penetration depth, area type, and the tip
    global area
    global trunc_index
    a1 = list() #placeholder for the penetrated area at that specific depth
    d1 = dep*100 #penetration depth array, in cm
    #print(len(d1))
    r = list() #placeholder for the radius at that specific depth
    if tiptype == 'c':
        if atype == 'm':
            for k in range(0,len(d1)):
                if d1[k]<length:
                    r.append(.22+d1[k]*((4.375-.22)/7.55))
                    #r.append(d1[k]*np.tan(30*pi/180)) this is from the original matlabcode
                    a1.append(pi*r[k]*(sqrt((r[k]**2)+(d1[k]**2))))
                elif d1[k]>=length:
                    r.append(4.375)
                    a1.append(pi*r[k]*(sqrt((r[k]**2)+(length**2))))
                a1[k] = a1[k]/10000
            area = a1
            #print(r)
            print(np.column_stack((d1,r, area)))
        elif atype == 'p':
            for k in range(0,len(d1)):
                if d1[k]<length:
                    r.append(.22+d1[k]*((4.375-.22)/7.55))
                    # r.append(d1[k]*np.tan(30*pi/180))
                    a1.append(pi*r[k]**2)
                elif d1[k]>=length:
                    r.append(4.375)
                    if r[k-1]<r[k]:
                        trunc_index = r.index(r[k-1])+jindex+dropstart #performs analysis only on readings beyond depth of cone height
                    a1.append(pi*(r[k])**2)
                a1[k] = a1[k]/10000
            area = a1

    elif tiptype == 'b':
        if atype =='m':
            for k in range(0,len(d1)):
                if d1[k]<length:
                    r.append(4.375)
                    a1.append(pi*r[k]**2 + 2*pi*r[k]*d1[k])
                if d1(k)>=length:
                    r.append(4.375)
                    a1.append(pi*r[k]**2 + 2*pi*r[k]*length)
                a1[k]=a1[k]/10000
            area = a1
        elif atype == 'p':
            for k in range(0,len(d1)):
                a1.append(pi*4.375**2)
                a1[k]=a1[k]/10000
            area = a1

    '''elif tiptype == "p":
        if atype == 'm':
            for k in range(1,len(d)):
                if d1[k]<length:
                     r[k]=sqrt(2.4184*d1[k])
                           polarfun = @(theta,r) r.*sqrt(0.745*r.^2 + 1);
                           A1(k)= integral2(polarfun,0,2*pi,0,r(k));
            
        elif atype == 'p':
            for k in range(1,len(d)):'''

def qdynfun(acc): # calculates dynamic bearing capacity
    global qdyn
    global maxbcdep
    global maxqdyn
    global qdyntable

    buoy = 1020*0.002473
    if droptype == "w": #water drops
        Fbe = (mass-buoy)*acc #drop force

    elif droptype =="a": #air drops
        Fbe = mass*acc

    qdyn = (Fbe/area)/1000 #Dynamic bearing capacity (kPa)
    qdyntable = pd.DataFrame()
    qdyntable["Qdyn"] = qdyn
    qdyntable["Depth"] = dep
    qdyntablet = qdyntable#.truncate(before=trunc_index)
    qdynt = qdyntablet["Qdyn"]
    #print(qdyntable)

    maxqdyn = qdynt.max() #gives maximum qdyn
    maxbcdep = qdyntablet.loc[qdyntable["Qdyn"]==maxqdyn, 'Depth'].item()

    maxqdyn = round(maxqdyn,1)
    maxbcdep = round(maxbcdep,4)
    #print("maxbcdep = ", maxbcdep) #finds the penetration depth corresponding to the maximum qdyn


    """bcmin = argrelmin(qdyn)
    peaksArray = np.array(bcmin)
    print("Peaks Array = ", peaksArray)"""

def qsbcfun(acc): #calculates quasi-static bearing capacity
    global bctable
    global qsbc
    global maxHqsbc
    global maxLqsbc
    global maxAqsbc

    if droptype == "w":
        srcv = np.log10(vel/0.02) #Velocity portion of the strain rate correction. 
        srfk = [0.2, 0.4, 1, 1.5] #list of strain rate factors to run; must be in increasing order

        bctable = pd.DataFrame()
        for i in range(0,len(srfk)):
            fsr = 1+srfk[i]*srcv
            qsbc = qdyn/fsr
            bctable.insert(i, "qsbc for srf = "+str(srfk[i]), qsbc)

        bctable_avgs = pd.DataFrame()
        bctable_avgs["High Average qsbc"]= (bctable.iloc[:,0]+bctable.iloc[:,1])/2 #Average of the lowest 2 strain rate factors
        bctable_avgs["Low Average qsbc"] = (bctable.iloc[:,int(len(srfk))-1]+bctable.iloc[:,int(len(srfk)-2)])/2 #average of the highest 2 strain rate factors
        maxHqsbc = bctable_avgs["High Average qsbc"].max()
        maxHqsbc = round(maxHqsbc, 1)
        maxLqsbc = bctable_avgs["Low Average qsbc"].max()
        maxLqsbc = round(maxLqsbc,1)
        maxAqsbc = "-"

        #print(bctable)

    elif droptype == "a":
        srcv = np.log10(vel/0.02) #Velocity portion of the strain rate correction. 
        srfk = 0.31*maxvel
        fsr = 1+srfk*srcv
        qsbc = qdyn/fsr
        maxHqsbc = "-"
        maxLqsbc = "-"
        maxAqsbc = qsbc.max()
        maxAqsbc = round(maxAqsbc,1)

def dr(accg): #Albatal 2019 method for calculating relative density (Duck, NC specific!)
    global Dr
    maxacc = accg.max() #maximum deceleration, in g
    #print("Max deceleration = ", maxacc)
    Dr = -2.18*10**-4*maxacc**3+1.29*10**-2*maxacc**2+1.61*maxacc-13.09 #Albatal 2019
    Dr = round(Dr, 1)
    #print("Relative density is ", dr)

def ignore_drop():
    global keep_drop
    print('Keep Drop? (y/n)')
    keep_drop=input()

def duncan_correlation(): #duncan correlation for caluclating friction angle
    global phi
    global sign
    #coefficients A, B, C, and D used in the Duncan Correlation. Duncan[34,10,3,2], Albatal[34,10,2,5]
    A = 34
    B = 10
    C = 3
    D = 2
    pa = 101.325 #kPa
    print("max penetration depth = ", maxdep)
    if sign == "uk":
        sign = 1120*maxdep #1120 = 70pcf in kg/m^3

    phi = A + B*Dr/100-(C+D*Dr/100)*math.log(sign/pa)
    phi = round(phi, 1)
    print("phi =", phi)

def firmnessfactor(acc, drop):
    global maxacc
    global ff
    maxacc = acc.max()
    g = 9.80665
    tp = len(drop)/2000

    ff = maxacc/(g*tp*maxvel)
    ff = round(ff,1)
    print("Firmness factor = ", ff)

def bctruncate(dropg, drop, acc, qdyn):
    dropg = pd.DataFrame(dropg)
    drop = pd.DataFrame(drop)
    qdyn = pd.DataFrame(qdyn)
    acc = pd.DataFrame(acc)
    dropg = dropg.truncate(before=trunc_index)
    drop = drop.truncate(before=trunc_index)
    acc = acc.truncate(before=trunc_index)
    qdyn = qdyn.truncate(before=trunc_index)


In [6]:
#Plots for exporting
def overviewplot(): #Plot showing all accellerometers and pore pressure readings
    fig, (ax1, ax2, ax3) = plt.subplots(3)
    fig.set_size_inches(14, 7)
    #plt.tight_layout()
    ax1.plot(time, g250g, label="250g")
    ax1.plot(time, g50g, label="50g")
    ax1.plot(time, g18g, label="18g")
    ax1.plot(time, g2g, label="2g")
    ax1.legend(loc = "upper right")
    ax1.set(ylabel="Deceleration (g)")
    ax1.set_title("BD file "+fileNum)
    ax1.grid()

    ax2.plot(time, ppm, label="Pore Pressure")
    ax2.set(ylabel="Pore Pressure (kPa)")
    ax2.grid()

    ax3.plot(time, gX55g, label="X 55g")
    ax3.plot(time, gY55g, label="Y 55g")
    ax3.legend(loc = "upper right")
    ax3.set(ylabel="Deceleration (g)")
    ax3.set(xlabel="Time (s)")
    ax3.grid()

    fig.subplots_adjust(bottom=.1, left = .1)
    plotName = fileNum+" Overview.png"
    #plt.savefig(plotPath / plotName)
    plt.show()

def integplot(drop, accName): #Deceleration,Velocity,and penetration depth vs time plots
    fig, (ax1, ax2, ax3) = plt.subplots(3)
    fig.set_size_inches(7, 7)
    #resize to 7,7, 7,14
    #plt.tight_layout()
    timereset = drop["Time (s)"]- drop["Time (s)"].iloc[0]
    ax1.plot(timereset, drop[accName], color = "k" ) #,marker = 11
    ax1.set(ylabel="Deceleration (m/s^2)")
    ax1.set_title("BD file "+fileNum+ " Drop #"+str(a))
    ax1.grid()

    ax2.plot(timereset, drop['Velocity (m/s)'] , color = "k" )
    ax2.set(ylabel="Velocity (m/s)")
    ax2.grid()

    ax3.plot(timereset, drop["Penetration Depth (m)"] , color = "k")
    ax3.set(ylabel="Penetration Depth (m)", xlabel="Time(s)")
    ax3.grid()

    fig.subplots_adjust(bottom=.1, left = .1)
    plotName = fileNum+" "+str(a)+" Integ"
    #plt.savefig(plotPath / plotName)
    plt.show()

def dep_qsbc_comboplot(dropg, drop, accNameg, qdyn):
    fig, (ax1, ax2) = plt.subplots(1,2)
    fig.set_size_inches(7, 7)
    ax1.plot(dropg[accNameg], drop["Penetration Depth (m)"]*100, color = "k", linestyle = "solid") #marker = 11
    ax1.plot(drop["Velocity (m/s)"], drop["Penetration Depth (m)"]*100, color = "k", linestyle = "dashed")
    ax1.plot(drop["X55g (m/s^2)"], drop["Penetration Depth (m)"]*100, color = "b", linestyle = "dashed")
    ax1.plot(drop["Y55g (m/s^2)"], drop["Penetration Depth (m)"]*100, color = "r", linestyle = "dashed")
    ax1.set(xlabel="Deceleration (g) and Velocity (m/s)", ylabel="Penetration Depth (cm)")
    ax1.invert_yaxis()
    ax1.legend(["Deceleration (g)", "Velocity (m/s)", "X tilt (g)", "Y tilt (g)"], loc = "upper right")
    ax1.set_title("BD file "+fileNum)
    ax1.grid()

    if droptype == "a": #for partial saturation, only one qsbc curve
        ax2.plot(qdyn, drop["Penetration Depth (m)"]*100, label="Qdyn") #color = "k", marker = 11 #Plots Qdyn
        ax2.plot(qsbc, drop["Penetration Depth (m)"]*100, label="QSBC") #plots QSBC
        ax2.set(xlabel="Bearing Capacity (kPa)")
        ax2.set_xlim(0,800)
        ax2.invert_yaxis()
        ax2.legend(["Qdyn", "QSBC"], loc = "upper right")
        ax2.set_title("Drop #"+str(a))
        ax2.grid()

    else: #for saturated drops, use a range of strain-rate factors
        ax2.plot(qdyn, drop["Penetration Depth (m)"]*100, label="Qdyn") #color = "k", marker = 11 #Plots Qdyn
        ax2.plot(bctable.iloc[:,0], drop["Penetration Depth (m)"]*100, label=str(bctable.columns[0])) #plots QSBC with smallest srf
        ax2.plot(bctable.iloc[:,len(bctable.columns)-1], drop["Penetration Depth (m)"]*100, label=str(bctable.columns[len(bctable.columns)-1])) #plots QSBC with largest srf
        for i in range(1,len(bctable.columns)-1): #plots qsbc for all other srfs
            ax2.plot(bctable.iloc[:,i], drop["Penetration Depth (m)"]*100, label=str(bctable.columns[i]), color = '.6')
        ax2.set(xlabel="Bearing Capacity (kPa)")
        ax2.set_xlim(0,)
        ax2.invert_yaxis()
        ax2.legend(["Qdyn", str(bctable.columns[0]), str(bctable.columns[len(bctable.columns)-1])], loc = "upper right")
        ax2.set_title("Drop #"+str(a))
        ax2.grid()
    
    fig.subplots_adjust(bottom=.1, left = .1)
    



#plots for troubleshooting 
def depthplot(dropg, drop, accNameg): #Velocity and develeration vs. penetration depth 
    fig, (ax1) = plt.subplots(1)
    ax1.plot(dropg[accNameg], drop["Penetration Depth (m)"]*100, color = "k", linestyle = "solid") #marker = 11
    ax1.plot(drop["Velocity (m/s)"], drop["Penetration Depth (m)"]*100, color = "k", linestyle = "dashed")
    ax1.set(xlabel="Deceleration (g) and Velocity (m/s)", ylabel="Penetration Depth (cm)")
    ax1.invert_yaxis()
    ax1.legend(["Deceleration (g)", "Velocity (m/s)"])
    ax1.set_title("BD file "+fileNum)
    plt.show()

def peakplot(): # Plot showing peak deceleration
    peakplot = plt.plot(x)
    peakplot = plt.plot(peaks, x[peaks], "x")
    plt.show()

def qdynplot(drop, qdyn): #Plot showing dynamic bearing capacity vs depth
    #print(len(drop))
    #print(len(qdyn))
    #print((bctable.iloc[:,0]))
    #print(len(bctable.iloc[:,len(bctable.columns)-1]))
    fig, (ax1) = plt.subplots(1)

    if droptype == "a": #for partial saturation, only one qsbc curve
        ax1.plot(qdyn, drop["Penetration Depth (m)"]*100, label="Qdyn") #color = "k", marker = 11 #Plots Qdyn
        ax1.plot(qsbc, drop["Penetration Depth (m)"]*100, label="QSBC") #plots QSBC
        ax1.set(xlabel="Bearing Capacity (kPa)")
        ax1.set_xlim(0,)
        ax1.invert_yaxis()
        ax1.legend(["Qdyn", "QSBC"])
        ax1.set_title("Bearing Capacity- "+fileNum+ " "+str(a))
        ax1.grid()

    else:
        ax1.plot(qdyn, drop["Penetration Depth (m)"]*100, label="Qdyn") #color = "k", marker = 11 #Plots Qdyn
        ax1.plot(bctable.iloc[:,0], drop["Penetration Depth (m)"]*100, label=str(bctable.columns[0])) #plots QSBC with smallest srf
        ax1.plot(bctable.iloc[:,len(bctable.columns)-1], drop["Penetration Depth (m)"]*100, label=str(bctable.columns[len(bctable.columns)-1])) #plots QSBC with largest srf
        for i in range(1,len(bctable.columns)-1): #plots qsbc for all other srfs
            ax1.plot(bctable.iloc[:,i], drop["Penetration Depth (m)"]*100, label=str(bctable.columns[i]), color = "k")
        ax1.set(xlabel="Bearing Capacity (kPa)", ylabel="Penetration Depth (cm)")
        ax1.set_xlim(0,)
        ax1.invert_yaxis()
        ax1.legend(["Qdyn", str(bctable.columns[0]), str(bctable.columns[len(bctable.columns)-1])])
        ax1.set_title("Bearing Capacity- "+fileNum+ " "+str(a))
    
    plt.show()

def tiltplot():
    fig, (ax1) = plt.subplots(1)
    plt.plot(time, gX55g, label="2g")
    plt.plot(time, gY55g, label="2g")
    ax1.legend()
    ax1.set(ylabel="Deceleration (g)")
    ax1.set(xlabel="Time (s)")
    ax1.set_title("BD file "+fileNum)


In [4]:
from matplotlib.font_manager import FontProperties
font = FontProperties()
font.set_name('Calibri')

def duck_dep_qsbc_comboplot(dropg, drop, dropt, accNameg, qdynt):
    font = {'fontname': 'Calibri'}
    fig, (ax1, ax2) = plt.subplots(1,2)
    fig.set_size_inches(7, 7)
    ax1.plot(dropg[accNameg], drop["Penetration Depth (m)"]*100, color = "k", linestyle = "solid") #marker = 11
    ax1.plot(drop["Velocity (m/s)"], drop["Penetration Depth (m)"]*100, color = "k", linestyle = "dashed")
    #ax1.plot(drop["X55g (m/s^2)"], drop["Penetration Depth (m)"]*100, color = "b", linestyle = "dashed")
    #ax1.plot(drop["Y55g (m/s^2)"], drop["Penetration Depth (m)"]*100, color = "r", linestyle = "dashed")
    ax1.set(xlabel="Deceleration (g) and Velocity (m/s)", ylabel="Penetration Depth (cm)", FontProperties = font)
    ax1.set_xlim(left = 0)
    ax1.set_ylim(0,dep.max()*100+.25)
    ax1.invert_yaxis()
    ax1.legend(["Deceleration (g)", "Velocity (m/s)"], FontProperties = font)
    ax1.set_title("BD file "+fileNum, FontProperties = font)
    ax1.grid()

    
    if droptype == "a": #for partial saturation, only one qsbc curve
        y = dropt["Penetration Depth (m)"]*100
        ax2.plot(qdynt[10:-50], y[10:-50], label="Qdyn") #color = "k", marker = 11 #Plots Qdyn, chops off first 40 readings
        ax2.plot(qsbc[10:-50], y[10:-50], label="QSBC") #plots QSBC; chops off last 10 points which go off towards infinity (can adjust as needed)
        ax2.set(xlabel="Bearing Capacity (kPa)",)
        ax2.set_xlim(0,qdyn.max()+10)
        ax2.set_ylim(0,dep.max()*100+.25)
        ax2.invert_yaxis()
        ax2.legend(["Qdyn", "QSBC"], loc = "upper right")
        ax2.set_title("Drop #"+str(a))
        ax2.grid()

    else: #for saturated drops, use a range of strain-rate factors
        y = list(dropt["Penetration Depth (m)"]*100)
        ax2.plot(qdynt[10:-20], y[10:-20], label="Qdyn") #color = "k", marker = 11 #Plots Qdyn
        ax2.plot(bctablet.iloc[10:-20,0], y[10:-20], label=str(bctablet.columns[0])) #plots QSBC with smallest srf
        ax2.plot(bctablet.iloc[10:-20,len(bctable.columns)-1], y[10:-20], label=str(bctable.columns[len(bctable.columns)-1])) #plots QSBC with largest srf
        for i in range(1,len(bctablet.columns)-1): #plots qsbc for all other srfs
            ax2.plot(bctablet.iloc[10:-20,i], y[10:-20], label=str(bctablet.columns[i]), color = '.6')
            #bctablet.iloc[:,i].pop()
        ax2.set(xlabel="Bearing Capacity (kPa)")
        ax2.set_xlim(0,)
        ax2.invert_yaxis()
        ax2.legend(["Qdyn", str(bctablet.columns[0]), str(bctablet.columns[len(bctablet.columns)-1])], loc = "upper right")
        ax2.set_title("Drop #"+str(a))
        ax2.grid()

    #fig.subplots_adjust(bottom=.1, left = .1)
    plotName = fileNum+" "+str(a)+" BC"
    #plt.savefig(plotPath / plotName)
    plt.show()

In [7]:
#Exporting Functions
def exporttoexcel():
    with pd.ExcelWriter(outputPath, mode="a", if_sheet_exists='replace') as writer:
        output_table.to_excel(writer, sheet_name = fileNum, index=False)

def troubleshooting_export():
        with pd.ExcelWriter(troubleshootingPath, mode="a", if_sheet_exists = 'new') as writer:
            drop1.to_excel(writer, sheet_name = str(a), index = False)
            qdyntable.to_excel(writer, sheet_name = "qdyn", index = False)


Bring in data and calibrate

In [5]:

#Set the penetrometer mass and length
masslength(tiptype)

# READ BD DATA IN
data_array = []  # creates an empty array for us to fill with bd data
fileName = "bLog"+fileNum+".bin"
# print(fileName)
newPath = binFilepath / fileName
#print(newPath)
file = open(newPath, 'rb')  # read file
element = file.read(3)  # create a byte list with each element having 3 bytes

while element:
    # Convert to signed integer before adding to data array
    iVAl = int.from_bytes(element, byteorder='big', signed=True)
    data_array.append(iVAl)  # adds the reshaped data from the bd file to the data frame
    element = file.read(3)

np_array = np.array(data_array)  # create numpy array from the list
np_array = np.reshape(np_array, (-1, 10))  # convert the 1d array to 2d array with 10 cols

#print(np_array.shape)
# print(np_array)

df = pd.DataFrame(np_array) # Creates a Dataframe in pandas from the bd data
df.columns = ['Count', 'no clue', 'g2g', 'g18g', 'g50g', 'ppm', 'g200g', 'gX55g', 'gY55g', 'g250g'] # names columns
# print(dfCal)

# APPLY CALIBRATION FACTORS
if BD == 3:  # calibration factors from March 2023
    g2g = (df['g2g']-33570.1)/1614577.9 #- offset# accelerometers are in g
    g18g = (df['g18g']+13495)/163387.2 #- offset
    g50g = (df['g50g']-238817.4)/63779.5 #- offset
    ppm = ((df['ppm']-139040.1)/20705) * 6.89475729 # converts to kPa
    g200g = (df['g200g'] -262332.4)/38888.7 #- offset
    gX55g = (df['gX55g']-70406)/59754.3
    gY55g = (df['gY55g']-69421.1)/141871.5
    g250g = (df['g250g']-39077.4)/13746.9 #- offset

if BD == 2: # calibration factors from March 2023
    g2g = (df['g2g']+36597.6)/1637627.3 #- offset# accelerometers are in g
    g18g = (df['g18g']-26185.3)/160297.2 #- offset
    g50g = (df['g50g']-212256.9)/63968.7 #- offset
    ppm = ((df['ppm']+55518.9)/18981.7) * 6.89475729 # converts to kPa
    g200g = (df['g200g']-175499.4)/30583.8 #- offset
    gX55g = (df['gX55g']-53629.9)/68590.9
    gY55g = (df['gY55g']-43694.3)/68280.3
    g250g = (df['g250g']-39619.9)/13545.8 #- offset

if BD == 1: # calibration factors from March 2023
    g2g = (df['g2g']-43727.6)/1625064 #- offset # accelerometers are in g
    g18g = (df['g18g']-45085.6)/160925.7 #- offset
    g50g = (df['g50g']-173493.4)/63944.6 #- offset
    ppm = ((df['ppm']+31776.1)/20679.7) * 6.89475729 # this is kPa
    g200g = (df['g200g'] -731734.3)/32192.4  #- offset
    gX55g = (df['gX55g'] -68837.9)/52137.3
    gY55g = (df['gY55g']-68015.9)/28074.9
    g250g = (df['g250g']+10580)/13688.7 #- offset

time = (df['Count']-df['Count'].iloc[0]+1)/2000 # gives time in s
count = df["Count"]-df['Count'].iloc[0]

# make a new dataframe of the calibrated values in units of g
dfCalg = pd.DataFrame([count, time, g2g, g18g, g50g, g200g, g250g, gX55g, gY55g, ppm])
dfCalg = dfCalg.T
dfCalg.columns = ['Count', 'Time (s)', '2g (g)', '18g (g)', '50g (g)', '200g (g)', '250g (g)', 'X55g (g)', 'Y55g (g)', 'Pore Pressure (kPa)']  # names columns
#print(dfCalg)

#make a new dataframe of the calibrated values in units of m/s^2
dfCal = pd.DataFrame([count, time, g2g, g18g, g50g, g200g, g250g, gX55g, gY55g, ppm])
dfCal = dfCal.T
dfCal.columns = ['Count','Time (s)', '2g (m/s^2)', '18g (m/s^2)', '50g (m/s^2)', '200g (m/s^2)', '250g (m/s^2)', 'X55g (m/s^2)', 'Y55g (m/s^2)', 'Pore Pressure (kPa)']  # names columns
dfCal['2g (m/s^2)'] = dfCal['2g (m/s^2)'] * 9.80665
dfCal['18g (m/s^2)'] = dfCal['18g (m/s^2)'] * 9.80665
dfCal['50g (m/s^2)'] = dfCal['50g (m/s^2)'] * 9.80665
dfCal['200g (m/s^2)'] = dfCal['200g (m/s^2)'] * 9.80665
dfCal['250g (m/s^2)'] = dfCal['250g (m/s^2)'] * 9.80665
dfCal['X55g (m/s^2)'] = dfCal['X55g (m/s^2)'] * 9.80665
dfCal['Y55g (m/s^2)'] = dfCal['Y55g (m/s^2)'] * 9.80665
#print(dfCal)

locate drops

In [8]:

#Locate the drops
x = np.array(g250g)  # what accelerometer to get the peaks from - use 250 because it will never be maxed out
max250 = max(g250g)

peaks, _ = find_peaks(x, height = 2, distance=1000, prominence=3)  # finds the largest peaks more than 2g spaced at least 10000 counts apart

peaksArray = np.array(peaks)  # prints a list of the count where the peaks occur
#peaksArray = filter(filterpeak, peaksArray)

#print(peaksArray)
q = (peaksArray.shape) #gives number of peaks
nDrops = int(q[0]) #number of drops in the file
#print(nDrops)

# For each drop, find the start and end points and integrate to solve for velocity and acceleration
#peakplot()



peakplot()
#overviewplot()

In [10]:
a = 1 #what drop do you want to analyze? USER INPUT
a = a-1

peak = int(peaksArray[a])  # count at the ath drop
print("peak index:" , peak)
dropstartend(peak) #zooms in the drop file to only consider 500 counts before and 1500 counts after the peak deceleration
#print(dropstart, dropend)

drop1 = dfCal[dropstart:dropend]  # start and end points of the drop in m/s^2
drop1g = dfCalg[dropstart:dropend]  # start and end points of the drop in g
drop1 = pd.DataFrame(drop1)  # makes dataframe including all data within the start and end points of the drop
drop1g = pd.DataFrame(drop1g)
dg = drop1g 
d = drop1
accPick(dg, d)  # chooses what accelerometer to use
acc1 = acc
acc1g = accg
acc1Name = accName
acc1Nameg = accNameg
#findchangepts() #automatically identifies the moment of penetration


peak index: 28061


In [11]:

peakpick()

start of penetration:  28024


In [81]:
finddropend(a) #automatically find the end of the drop
print("penend: ", penend)
offset = acc1g[penstart] #sets the offset to be the acceleration at the moment of impact
print("offset: ", offset)
#print(drop)
dg = dfCalg[penstart:penend] #chops the file to only include penetration 
d = dfCal[penstart:penend]
acc1g = accg[penstart:penend]
acc1g = acc1g-offset
acc1 = acc[penstart:penend]
offsetms = offset * 9.80665
acc1 = acc1-offsetms


drop1 = d
drop1g = dg
integration(d, acc1) #double integration to solve for velocity and penetration depth
drop1 = d
areafind() #area calculations for bearing capacity calculations
qdynfun(acc1) #calculation of dynamic bearing capacity
qsbcfun(acc1) #calculation of quasi-static bearing capacities
qdyn1 = qdyn
integplot(drop1,acc1Name)
#qdynplot(drop1, qdyn1)
print("Max penetration depth is: ", maxdep)
print("Pre-truncation d1", drop1)
drop1 = pd.DataFrame(drop1)
drop1g = pd.DataFrame(drop1g)
acc1 = pd.DataFrame(acc1)
qdyn1 = pd.DataFrame(qdyn1)
if droptype == "w":
    bctable = pd.DataFrame(bctable)

drop1t = drop1 #.truncate(before=trunc_index) #this truncates each df such that only depths below cone height are considered
drop1gt = drop1g #.truncate(before=trunc_index)
acc1t = acc1 #.truncate(before=trunc_index)
acc1t = np.array(acc1t)
acc1gt = acc1g #.truncate(before=trunc_index)
acc1gt = np.array(acc1gt)
drop1t = drop1 #truncate(before=trunc_index)
qdyn1t = qdyn1#.truncate(before=trunc_index)
if droptype == "w":
    bctablet = bctable#.truncate(before=trunc_index)

#bctruncate(drop1g, drop1, acc1, qdyn1)
firmnessfactor(acc1t, drop1) #calculates the firmness factor
if soiltype == "s":
    dr(acc1gt) #calculates relative density
    duncan_correlation() #calculates friction angle
elif soiltype == "u":
    if maxdep <= .2:
        dr(acc1gt)
        duncan_correlation()
pentime=(penend - penstart)/2 #ms #calculates the time of penetration

dep_qsbc_comboplot(drop1g,drop1,acc1Nameg, qdyn1)
plt.show()
ignore_drop()
if keep_drop == "y":
    duck_dep_qsbc_comboplot(drop1g, drop1, drop1t, acc1Nameg, qdyn1t)
    print("pentime= ", pentime)

    output_table1 = pd.DataFrame([BD, fileNum, a, soiltype, atype, tiptype, droptype, maxdep*100, maxacc, maxvel, maxbcdep*100, maxqdyn, maxHqsbc, maxLqsbc, maxAqsbc, ff, pentime, Dr, phi])
    output_table1 = output_table1.T
    output_table = output_table1
    output_table.columns = ["BlueDrop", "BD File", "Drop No.", "Soil Type", "Area Type", "Tip Type", "Drop Type", "Penetration Depth (cm)", "Maximum Deceleration (m/s^2)", "Impact Velocity (m/s)", "Depth of Max QSBC (cm)", "Max Qdyn (kPa)", "Max QSBC-Upper Bounds (kPa)", "Max QSBC-Lower Bounds (kPa)", "Max QSBC-Average (kPa)", "Firmness Factor (m-1)", "Penetration Time (ms)", "Relative Density (%)", "Friction Angle (deg.)"]  # Creates an empty Dataframe in pandas from the output data
else:
    dep_qsbc_comboplot(drop1g,drop1,acc1Nameg, qdyn1)
    plotName = fileNum+" "+str(a)+" BC - REJECTED"
    plt.savefig(plotPath / plotName)
    output_table1 = pd.DataFrame([BD, fileNum, a, soiltype, atype, tiptype, droptype,'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN'])
    output_table1 = output_table1.T
    output_table = output_table1
    output_table.columns = ["BlueDrop", "BD File", "Drop No.", "Soil Type", "Area Type", "Tip Type", "Drop Type", "Penetration Depth (cm)", "Maximum Deceleration (m/s^2)", "Impact Velocity (m/s)", "Depth of Max QSBC (cm)", "Max Qdyn (kPa)", "Max QSBC-Upper Bounds (kPa)", "Max QSBC-Lower Bounds (kPa)", "Max QSBC-Average (kPa)", "Firmness Factor (m-1)", "Penetration Time (ms)", "Relative Density (%)", "Friction Angle (deg.)"]  # Creates an empty Dataframe in pandas from the output data


penend:  64989
offset:  0.0
[[0.00000000e+00 2.20000000e-01 1.52053084e-05]
 [1.47248589e-01 3.01035482e-01 3.16932046e-05]
 [2.93532681e-01 3.81540171e-01 5.77012654e-05]
 [4.38084895e-01 4.61091753e-01 9.21317591e-05]
 [5.80000723e-01 5.39192451e-01 1.34144303e-04]
 [7.18323948e-01 6.15316027e-01 1.82836669e-04]
 [8.52172394e-01 6.88976993e-01 2.37195082e-04]
 [9.80844224e-01 7.59789106e-01 2.96148468e-04]
 [1.10382817e+00 8.27471000e-01 3.58623365e-04]
 [1.22072637e+00 8.91803719e-01 4.23553725e-04]
 [1.33120009e+00 9.52600846e-01 4.89881597e-04]
 [1.43497379e+00 1.00971074e+00 5.56580655e-04]
 [1.53185183e+00 1.06302574e+00 6.22688185e-04]
 [1.62171550e+00 1.11248052e+00 6.87324157e-04]
 [1.70450408e+00 1.15804165e+00 7.49694332e-04]
 [1.78020420e+00 1.19970178e+00 8.09093652e-04]
 [1.84884215e+00 1.23747538e+00 8.64907878e-04]
 [1.91047506e+00 1.27139389e+00 9.16611196e-04]
 [1.96519393e+00 1.30150739e+00 9.63771454e-04]
 [2.01313002e+00 1.32788811e+00 1.00605745e-03]
 [2.05445293

C:\Users\elise\AppData\Local\Temp\ipykernel_26304\2952692391.py:275: RuntimeWarning: divide by zero encountered in log10
  srcv = np.log10(vel/0.02) #Velocity portion of the strain rate correction.
C:\Users\elise\AppData\Local\Temp\ipykernel_26304\2952692391.py:275: RuntimeWarning: invalid value encountered in log10
  srcv = np.log10(vel/0.02) #Velocity portion of the strain rate correction.


Max penetration depth is:  0.0218
Pre-truncation d1          Count  Time (s)  2g (m/s^2)  18g (m/s^2)  50g (m/s^2)  200g (m/s^2)  \
64915  64915.0   32.4580   10.364800    10.014808   -37.606040    -44.381730   
64916  64916.0   32.4585   10.651060    10.286805   -37.432653    -44.095712   
64917  64917.0   32.4590   11.566406    11.226008   -37.014594    -43.465958   
64918  64918.0   32.4595   12.300198    11.865991   -36.347262    -42.915405   
64919  64919.0   32.4600   12.220535    11.808422   -35.526473    -42.829150   
...        ...       ...         ...          ...          ...           ...   
64984  64984.0   32.4925   19.781746   147.322909   144.427755     70.443785   
64985  64985.0   32.4930   19.781297   109.785541   104.877040     39.510823   
64986  64986.0   32.4935   19.781746    79.014942    67.639834     13.834893   
64987  64987.0   32.4940   19.781332    58.460689    36.943951     -5.796812   
64988  64988.0   32.4945   19.781129    38.658312    12.791691    -1

FileNotFoundError: [Errno 2] No such file or directory: 'H:\\.shortcut-targets-by-id\\1aF9t2aiRGWTftJMZFAOBixqvQniFBjnb\\Duck  2023\\Data\\Intertidal\\BlueDrop, Samples & Moisture Gage\\07March23\\BD2 3.8.23 - Drops Only\\Analysis Figures\\004D 0 BC - REJECTED.png'